In [1]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.5/599.5 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.

In [2]:
# Wait 60 seconds before connecting using these details, or login to https://console.neo4j.io to validate the Aura Instance is available
NEO4J_URI="neo4j+s://6ed9b7da.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="14V4z2NxsI8laZL3tvaAvyJzIdgyZCwgxKHbRWObatM"


In [3]:
import os
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [4]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [5]:
graph

In [21]:
groq_api_key="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"

In [22]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7c51ff268880>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7c51ff269540>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [23]:
!pip install --upgrade --quiet langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 3.5 MB/s eta 0:00:00


In [24]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

In [17]:
### Load the dataset of movie

movie_query="""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/Adityak8340/docbot/main/cleaned_final_data_v2.csv' AS row

MERGE (c:CVE {id: row.`CVE-ID`})
SET c.cvss_v3 = toFloat(row.`CVSS-V3`),
    c.cvss_v2 = toFloat(row.`CVSS-V2`),
    c.severity = row.SEVERITY,
    c.rip = row.RIP,
    c.cwe_id = row.`CWE-ID`,
    c.operating_system = row.Operating_System,
    c.software_component = row.Software_Component,
    c.version = row.Version,
    c.impact = row.Impact,
    c.affected_hardware = row.Affected_Hardware,
    c.network_requirements = row.Network_Requirements,
    c.affected_protocols = row.Affected_Protocols,
    c.authentication_required = row.Authentication_Required,
    c.privileges_required = row.Privileges_Required,
    c.user_interaction_required = row.User_Interaction_Required,
    c.vendor = row.Vendor

// Optionally, if there are relationships (e.g., linking vulnerabilities with protocols, software components, etc.)
FOREACH (protocol IN split(row.Affected_Protocols, '|') |
    MERGE (p:Protocol {name: trim(protocol)})
    MERGE (c)-[:AFFECTS_PROTOCOL]->(p))
    """


In [18]:
graph

In [19]:
graph.query(movie_query)

[]

In [20]:
graph.refresh_schema()
print(graph.schema)

Node properties:
CVE {id: STRING, cvss_v2: FLOAT, severity: STRING, cwe_id: STRING, operating_system: STRING, software_component: STRING, impact: STRING, vendor: STRING, network_requirements: STRING, affected_protocols: STRING, version: STRING, authentication_required: STRING, user_interaction_required: STRING, privileges_required: STRING, affected_hardware: STRING, cvss_v3: FLOAT}
Protocol {name: STRING}
Relationship properties:

The relationships:
(:CVE)-[:AFFECTS_PROTOCOL]->(:Protocol)


In [26]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True, allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7c5231856560>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7c51ff268880>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7c51ff269540>, model_name='Gemma2-9b-I

In [27]:
response=chain.invoke({"query":"CVE-2004-0167 having what attacks and Operating system?"})

response




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:CVE {id: 'CVE-2004-0167'})-[:AFFECTS_PROTOCOL]->(p:Protocol)
RETURN c.operating_system, p.name 

Full Context:
[]

> Finished chain.


{'query': 'CVE-2004-0167 having what attacks and Operating system?',
 'result': "I don't know the answer. \n"}

In [29]:
response=chain.invoke({"query":"Find vulnerabilities related to the ip_input.c software component."})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:CVE {software_component: 'ip_input.c'}) RETURN c

Full Context:
[{'c': {'severity': 'MEDIUM', 'cwe_id': 'CWE-20', 'cvss_v2': 5.0, 'impact': 'denial of service (crash or hang)', 'operating_system': 'BSD-derived', 'id': 'CVE-1999-0001', 'software_component': 'ip_input.c', 'network_requirements': 'remote', 'affected_protocols': 'TCP/IP'}}]

> Finished chain.


{'query': 'Find vulnerabilities related to the ip_input.c software component.',
 'result': 'CVE-1999-0001 is a vulnerability related to the ip_input.c software component. \n'}

In [30]:
response=chain.invoke({"query":"Show me vulnerabilities that affect the TCP/IP protocol."})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:CVE)-[:AFFECTS_PROTOCOL]->(p:Protocol) WHERE p.name = 'TCP/IP' RETURN c
Full Context:
[{'c': {'severity': 'MEDIUM', 'cwe_id': 'CWE-20', 'cvss_v2': 5.0, 'impact': 'denial of service (crash or hang)', 'operating_system': 'BSD-derived', 'id': 'CVE-1999-0001', 'software_component': 'ip_input.c', 'network_requirements': 'remote', 'affected_protocols': 'TCP/IP'}}, {'c': {'severity': 'HIGH', 'cwe_id': 'NVD-CWE-Other', 'vendor': 'Foundry Networks', 'cvss_v2': 7.5, 'impact': 'spoof or hijack sessions', 'affected_hardware': 'ServerIron switches', 'id': 'CVE-2000-0178', 'software_component': 'ServerIron switches', 'network_requirements': 'remote', 'affected_protocols': 'TCP/IP'}}, {'c': {'severity': 'LOW', 'cwe_id': 'NVD-CWE-Other', 'vendor': 'Microsoft', 'cvss_v2': 2.1, 'impact': 'denial of service', 'operating_system': 'Windows', 'id': 'CVE-2000-0232', 'software_component': 'TCP/IP Printing Services, aka Print Services for 

{'query': 'Show me vulnerabilities that affect the TCP/IP protocol.',
 'result': 'CVE-1999-0001, CVE-2000-0178, CVE-2000-0232, CVE-2003-0364, CVE-2004-0525, CVE-2012-0094, CVE-2012-0109  \n'}

In [31]:
response=chain.invoke({"query":"Show me vulnerabilities related to Microsoft products."})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:CVE) WHERE c.vendor = 'Microsoft'
RETURN c  

Full Context:
[{'c': {'severity': 'MEDIUM', 'cwe_id': 'NVD-CWE-Other', 'vendor': 'Microsoft', 'cvss_v2': 5.0, 'impact': 'bypass access restrictions', 'operating_system': 'Windows', 'id': 'CVE-1999-0012', 'software_component': 'web servers', 'network_requirements': 'remote'}}, {'c': {'severity': 'HIGH', 'cwe_id': 'NVD-CWE-Other', 'vendor': 'Microsoft', 'cvss_v2': 10.0, 'impact': 'read and delete shares', 'operating_system': 'Windows', 'id': 'CVE-1999-0119', 'software_component': 'NT 4.0 beta'}}, {'c': {'severity': 'HIGH', 'cwe_id': 'NVD-CWE-Other', 'user_interaction_required': 'Yes', 'vendor': 'Microsoft', 'cvss_v2': 7.5, 'impact': 'remote command execution', 'operating_system': 'Windows', 'id': 'CVE-1999-0280', 'software_component': 'Microsoft Internet Explorer', 'network_requirements': 'remote'}}, {'c': {'severity': 'HIGH', 'cwe_id': 'NVD-CWE-Other', 'user_interaction_

{'query': 'Show me vulnerabilities related to Microsoft products.',
 'result': 'Here are the vulnerabilities related to Microsoft products:\n\n* **CVE-1999-0012:**  A MEDIUM severity vulnerability in web servers for Windows that allows bypass access restrictions. \n* **CVE-1999-0119:** A HIGH severity vulnerability in NT 4.0 beta that allows read and delete shares.\n* **CVE-1999-0280:** A HIGH severity vulnerability in Microsoft Internet Explorer for Windows that allows remote command execution. User interaction is required.\n* **CVE-1999-0354:** A HIGH severity vulnerability in Internet Explorer, Word 97, and Outlook for Windows that allows arbitrary execution of Visual Basic programs. \n* **CVE-1999-0360:** A HIGH severity vulnerability in MS Site Server 2.0 with IIS 4 for Windows that allows code execution. This vulnerability requires remote access and exploits the HTTP protocol.\n* **CVE-1999-0364:** A HIGH severity vulnerability in Microsoft Access 97 for Windows that allows data 

In [32]:
response=chain.invoke({"query":"What are the details for CVE-1999-0013?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:CVE {id: 'CVE-1999-0013'}) RETURN c  

Full Context:
[{'c': {'severity': 'HIGH', 'cwe_id': 'NVD-CWE-Other', 'cvss_v2': 7.5, 'impact': 'other local users can access remote accounts', 'id': 'CVE-1999-0013', 'software_component': 'ssh-agent program', 'network_requirements': 'local', 'affected_protocols': 'SSH', 'authentication_required': 'Yes'}}]

> Finished chain.


{'query': 'What are the details for CVE-1999-0013?',
 'result': 'The CVE ID is CVE-1999-0013. The severity is HIGH with a CVSS v2 score of 7.5. It affects the ssh-agent program and if exploited, other local users can access remote accounts. Authentication is required. \n'}